In [43]:
import json

In [44]:
#load the affected queries

affected_queries_json = open("/home/manuel/Tesi/Codebase/ARC/files/affected_queries.json", "r")
affected_queries = json.load(affected_queries_json)
affected_queries = affected_queries.keys()
affected_queries = set(affected_queries)
print(f"Affected queries: {len(affected_queries)}")

#load the empty datasets and adjsut the name of the datasets

empty_datasets_json = open("/home/manuel/Tesi/Codebase/ARC/files/empty_datasets.json", "r")
empty_datasets = json.load(empty_datasets_json)

for i in range(len(empty_datasets)):
    empty_datasets[i] = empty_datasets[i].replace("dataset-", "")

empty_datasets = set(empty_datasets)
print(f"Number of empty datasets: {len(empty_datasets)}")

Affected queries: 210
Number of empty datasets: 5127


In [45]:
"""
@param path_run: string with the path of the run file
@return dict, key: affected query id, value: rank of the query (ordered list)
"""
def returnRankAffectedQueries(path_run: str) -> dict: 
    run = open(path_run, "r")

    ranks = dict()

    while True:
        line = run.readline()

        if not line:
            break
        
        split = line.split("\t")

        query_id = split[0]

        if query_id in affected_queries:
            rank = set()
            rank.add(split[2])
        
            while True:
                previous_position = run.tell()

                ln = run.readline()

                if not ln:
                    break

                splt = ln.split("\t")
                actual_query = splt[0]

                if actual_query == query_id:
                    rank.add(splt[2])
                else:
                    run.seek(previous_position)
                    break

            ranks[query_id] = rank 

    run.close()

    return ranks


In [46]:
#recover the ACORDAR and my ranks for every dataset-empty affected query

path_acordar_run = "/home/manuel/Tesi/Codebase/ARC/run/ACORDAR/BM25F.txt"
path_run = "/home/manuel/Tesi/Codebase/ARC/run/EDS/BM25[m+c].txt"

acordar_ranks = returnRankAffectedQueries(path_acordar_run)
our_ranks = returnRankAffectedQueries(path_run)

print(len(acordar_ranks.keys()))
print(len(our_ranks.keys()))


210
210


In [47]:
#from the previous extracted dict (affected query -> rank) for ACORDAR we extract from the rank only the list of empty datasets
#returned from ACORDAR

empty_datasets_in_acordar_queries = dict()

for query, rank in acordar_ranks.items():
    empty_datasets_returned = set.intersection(rank, empty_datasets)
    if len(empty_datasets_returned) > 0:
        print(f"ACORDAR returns empty datasets in query: {query}, datasets: {empty_datasets_returned}")
        empty_datasets_in_acordar_queries[query] = empty_datasets_returned



ACORDAR returns empty datasets in query: 3, datasets: {'25039', '25054'}
ACORDAR returns empty datasets in query: 22, datasets: {'11888', '2690', '43248', '9464'}
ACORDAR returns empty datasets in query: 28, datasets: {'24355'}
ACORDAR returns empty datasets in query: 80, datasets: {'88119', '88100'}
ACORDAR returns empty datasets in query: 82, datasets: {'32617', '32612'}
ACORDAR returns empty datasets in query: 99, datasets: {'3257'}
ACORDAR returns empty datasets in query: 108, datasets: {'11285'}
ACORDAR returns empty datasets in query: 112, datasets: {'3933', '63269', '29436', '3882', '11285', '63268'}
ACORDAR returns empty datasets in query: 134, datasets: {'55976', '85975'}
ACORDAR returns empty datasets in query: 162, datasets: {'24589', '84311'}
ACORDAR returns empty datasets in query: 175, datasets: {'11789'}
ACORDAR returns empty datasets in query: 194, datasets: {'24751'}
ACORDAR returns empty datasets in query: 206, datasets: {'82751', '84311', '28671', '28924', '83477', '

In [54]:
#for every query affected by empty datasets, return which datasets I don't return and ACORDAR does instead

not_returned_datasets = dict()

for query, empty_datasets_acordar in empty_datasets_in_acordar_queries.items():

    if not empty_datasets_acordar.issubset(our_ranks[query]):
        datasets = set()

        for empty_dataset in empty_datasets_acordar:
            if empty_dataset not in our_ranks[query]:
                datasets.add(empty_dataset)
        
        not_returned_datasets[query] = datasets

        print(f"Query: {query}, not returned: {datasets}")
 
print(f"Number of queries where i don't return empty datasets: {len(not_returned_datasets.keys())}")

Query: 22, not returned: {'43248', '2690', '11888', '9464'}
Query: 28, not returned: {'24355'}
Query: 80, not returned: {'88119', '88100'}
Query: 82, not returned: {'32617', '32612'}
Query: 99, not returned: {'3257'}
Query: 108, not returned: {'11285'}
Query: 112, not returned: {'3933', '63269', '29436', '3882', '11285', '63268'}
Query: 134, not returned: {'55976'}
Query: 162, not returned: {'24589', '84311'}
Query: 175, not returned: {'11789'}
Query: 194, not returned: {'24751'}
Query: 206, not returned: {'82751', '84311', '28924', '83477', '83174', '28732', '28744'}
Query: 1004, not returned: {'15786', '15779', '15771', '15753'}
Query: 1028, not returned: {'41454'}
Query: 1036, not returned: {'2206'}
Query: 1041, not returned: {'55459'}
Query: 1046, not returned: {'23924', '2165'}
Query: 1053, not returned: {'28622'}
Query: 1055, not returned: {'55200'}
Query: 1065, not returned: {'13349', '14509'}
Query: 1074, not returned: {'13290', '69094'}
Query: 1078, not returned: {'3786', '158

In [49]:
#visualize the most affected for my run, so the queries with a lot of empty datasets that are not returned
#by sorting the queries by the len of the list of the empty datasets not returned

not_returned_datasets_sorted = sorted(not_returned_datasets.items(), key=lambda x:len(x[1]), reverse= True)
print(not_returned_datasets_sorted)

[('1182', {'2079', '1998', '2048', '2171', '2270', '2124', '2187', '2065', '2567', '2263'}), ('1238', {'42856', '42845', '2987', '24005', '2945', '2243', '42796', '9460', '3108'}), ('10', {'85313', '83073', '84478', '83577', '82631', '82816', '85047', '82746'}), ('206', {'82751', '84311', '28924', '83477', '83174', '28732', '28744'}), ('1106', {'41092', '11831', '41132', '41136', '7542', '41073', '9776'}), ('224', {'72935', '14509', '72934', '13349', '72932', '72909', '72877'}), ('112', {'3933', '63269', '29436', '3882', '11285', '63268'}), ('1042', {'15743', '15779', '15762', '15786', '15788', '15771'}), ('1203', {'80845', '1103', '961', '969', '4053', '1859'}), ('66', {'43153', '2981', '43090', '3139', '3127', '3094'}), ('158', {'14735', '21569', '15250', '13815', '3106', '63268'}), ('188', {'84311', '28924', '10037', '83174', '84478', '28744'}), ('1160', {'2171', '2124', '2187', '2065', '2263'}), ('189', {'10037', '82596', '83612', '83073', '10036'}), ('37', {'43652', '3143', '44450

In [52]:
#calculate how many queries has 1 empty dataset not returned, 2 and so on

buckets = dict()

for query, empty_datasets_not_returned in not_returned_datasets_sorted:
    length = len(empty_datasets_not_returned)

    if length not in buckets.keys():
        buckets[length] = []

    buckets[length].append(query)

for bucket, query_list in buckets.items():
    print(f"Number of empty datasets not returned: {bucket} Number of queries: {len(query_list)}")


Number of empty datasets not returned: 10 Number of queries: 1
Number of empty datasets not returned: 9 Number of queries: 1
Number of empty datasets not returned: 8 Number of queries: 1
Number of empty datasets not returned: 7 Number of queries: 3
Number of empty datasets not returned: 6 Number of queries: 6
Number of empty datasets not returned: 5 Number of queries: 7
Number of empty datasets not returned: 4 Number of queries: 12
Number of empty datasets not returned: 3 Number of queries: 18
Number of empty datasets not returned: 2 Number of queries: 38
Number of empty datasets not returned: 1 Number of queries: 57


In [55]:
"""
@param trec_file file to a P@10 info file
@parma affected_queries set of queries ids (144 queries where I don't return some empty datasets)
@return dict: key: query id value: P@10
"""
def recoverP10FromTrec(trec_file, affected_queries):
    query_p10 = dict()

    while True:
        line = trec_file.readline()

        if not line:
            break

        split = line.split()
        query = split[1]
        p10 = split[2]

        if query in affected_queries:
            query_p10[query] = p10
    
    return query_p10
    

In [56]:
# recover for every empty datasets affected query the P@10 obtained from ACORDAR runs and from my runs

acordar_trec = open("/home/manuel/Tesi/Codebase/ARC/run/ACORDAR/BM25_P10.txt", "r")
run_trec = open("/home/manuel/Tesi/Codebase/ARC/run/EDS/BM25_P10.txt", "r")

acordar_P10 = recoverP10FromTrec(acordar_trec, not_returned_datasets.keys())
run_P10 = recoverP10FromTrec(run_trec, not_returned_datasets.keys())

comparison = dict()

for query, p10 in acordar_P10.items():
    features = dict()
    features["P10 (A)"] = float(p10)
    features["P10 (E)"] = float(run_P10[query])
    features["len"] = len(not_returned_datasets[query])
                    
    print(f"Query ID: {query} P10 (A): {p10} P10 (E): {run_P10[query]} Not returned empty datasets: {len(not_returned_datasets[query])}")
    comparison[query] = features

Query ID: 10 P10 (A): 0.7000 P10 (E): 0.0000 Not returned empty datasets: 8
Query ID: 1001 P10 (A): 0.1000 P10 (E): 0.0000 Not returned empty datasets: 2
Query ID: 1004 P10 (A): 0.7000 P10 (E): 0.6000 Not returned empty datasets: 4
Query ID: 1006 P10 (A): 0.6000 P10 (E): 0.1000 Not returned empty datasets: 1
Query ID: 1007 P10 (A): 1.0000 P10 (E): 0.4000 Not returned empty datasets: 2
Query ID: 1012 P10 (A): 0.3000 P10 (E): 0.2000 Not returned empty datasets: 2
Query ID: 1013 P10 (A): 0.6000 P10 (E): 0.4000 Not returned empty datasets: 2
Query ID: 1014 P10 (A): 1.0000 P10 (E): 0.7000 Not returned empty datasets: 2
Query ID: 1020 P10 (A): 0.2000 P10 (E): 0.0000 Not returned empty datasets: 2
Query ID: 1024 P10 (A): 0.3000 P10 (E): 0.0000 Not returned empty datasets: 5
Query ID: 1025 P10 (A): 0.1000 P10 (E): 0.0000 Not returned empty datasets: 3
Query ID: 1028 P10 (A): 0.8000 P10 (E): 0.6000 Not returned empty datasets: 1
Query ID: 1033 P10 (A): 1.0000 P10 (E): 0.2000 Not returned empty 

In [58]:
i = 0 
for query, features in comparison.items():
    recover = abs(features["P10 (A)"] - (features["P10 (E)"] + (features["len"] / 10 )))
    if recover < 0.2:
        print(f"Query ID: {query} P10 (A): {features['P10 (A)']} P10 (E): {features['P10 (E)']} Not returned empty datasets: {features['len']}")
        i+=1

print(i)

Query ID: 10 P10 (A): 0.7 P10 (E): 0.0 Not returned empty datasets: 8
Query ID: 1001 P10 (A): 0.1 P10 (E): 0.0 Not returned empty datasets: 2
Query ID: 1012 P10 (A): 0.3 P10 (E): 0.2 Not returned empty datasets: 2
Query ID: 1013 P10 (A): 0.6 P10 (E): 0.4 Not returned empty datasets: 2
Query ID: 1014 P10 (A): 1.0 P10 (E): 0.7 Not returned empty datasets: 2
Query ID: 1020 P10 (A): 0.2 P10 (E): 0.0 Not returned empty datasets: 2
Query ID: 1025 P10 (A): 0.1 P10 (E): 0.0 Not returned empty datasets: 3
Query ID: 1028 P10 (A): 0.8 P10 (E): 0.6 Not returned empty datasets: 1
Query ID: 1034 P10 (A): 0.3 P10 (E): 0.0 Not returned empty datasets: 1
Query ID: 1037 P10 (A): 0.8 P10 (E): 0.8 Not returned empty datasets: 1
Query ID: 1038 P10 (A): 1.0 P10 (E): 0.9 Not returned empty datasets: 1
Query ID: 1040 P10 (A): 0.7 P10 (E): 0.5 Not returned empty datasets: 1
Query ID: 1044 P10 (A): 0.8 P10 (E): 0.6 Not returned empty datasets: 2
Query ID: 1046 P10 (A): 0.8 P10 (E): 0.6 Not returned empty datase